### CMPN403 Programming Assignment
#### Regex to Minimized DFA Conversion
Team:
- Ahmed Tarek Abdellatif 1190157
- Mostafa Mohammad Mahmoud El-Nashar 1190212

#### TODOS:
- MinimizeDFA(FSM)-> FSM

#### Dependencies and Includes

In [ ]:
import sys
import os
if 'graphviz' not in sys.modules:
	if os.name == 'linux':
		!apt install graphviz
	%pip install graphviz

import json
import graphviz
import re
from typing import FrozenSet

#### State and FSM Classes
##### State is our state class, contains the following:
- A dictionary of transitions, where the key is the transition character and the value is the list of states to transition to
##### FSM is our base class for Finite State Machines, performs the following:
- Reads and writes to json files
- Creates graphviz graphs
- Creates an FSM from a literal string or single character
- Creates an FSM with no states (For use by subclasses)

In [ ]:
class State:
    def __init__(self, transitions: dict[str, list["State"]] = None):
        # {input : [list of states]}
        self.transitions: dict[str, list["State"]] = transitions if transitions is not None else dict()

    def __iter__(self):
        for key, value in self.transitions.items():
            yield (key, value)


class FSM:

    def __get_new_state_name(self) -> str:
        self.stateCounter += 1
        return "Q" + str(self.stateCounter).zfill(3)

    def __init__(self, literal: str = None, single_char: str = None):
        """
        *** Doesn't validate the literal string ***\n
        """
        self.nonTerminalStates: set[State] = set()
        self.terminalStates: set[State] = set()
        self.startingState: State = None
        self.stateToName: dict[State, str] = dict()
        self.nameToState: dict[str, State] = dict()
        self.stateCounter = -1
        if literal is not None: self.__literal_to_fsm(literal)
        elif single_char is not None: self.__single_char_to_fsm(single_char)

    def __name_to_state(self, name : str) -> State:
        """
        Returns the state with the given name\n
        Returns a new state if no state with the given name exists
        """
        if name in self.nameToState:
            return self.nameToState[name]
        # Assign a new state to the name
        self.nameToState[name] = State()
        # Update the reverse mapping
        self.stateToName[self.nameToState[name]] = name
        self.stateCounter = max(self.stateCounter, int(name[1:]))
        return self.nameToState[name]

    def __state_to_name(self, state : State) -> str:
        """
        Returns the name of the given state\n
        Returns a new name if no name is assigned to the given state
        """
        # Does the state already have a name?
        if state in self.stateToName:
            # Return the name
            return self.stateToName[state]
        # Assign a new name to the state
        self.stateToName[state] = self.__get_new_state_name()
        # Update the reverse mapping
        self.nameToState[self.stateToName[state]] = state
        return self.stateToName[state]

    def __literal_to_fsm(self, literal: str):
        """
        *** Doesn't validate the literal string ***\n
        Intializes the FSM to describe a literal string
        """
        if literal == "":
            self.startingState = State()
            self.terminalStates.add(self.startingState)
            return
        # Non-empty string
        previous_state = self.startingState = State()
        for char in literal:
            new_state = State()
            previous_state.transitions.update({char: [new_state]})
            self.nonTerminalStates.add(previous_state)
            previous_state = new_state
        self.terminalStates.add(previous_state)

    def __single_char_to_fsm(self, single_char: str):
        """
        Intializes the FSM to describe a single character
        """
        self.startingState = State({single_char: [State()]})
        self.nonTerminalStates.add(self.startingState)
        self.terminalStates.add(self.startingState.transitions[single_char][0])

    @staticmethod
    def read_JSON(JSONFILE: str) -> "FSM":
        """
        Takes JSON file name as string and reads an fsm from it
        """
        fsm = FSM()
        with open(JSONFILE) as f:
            file_contents = f.read()
            parsed_json: dict[str, str | dict[str, bool | dict[str, list[str]]]] = json.loads(file_contents)
            starting_state_name = parsed_json["@startingState"]
            parsed_json.pop("@startingState")
            for name, transitions in parsed_json.items():
                terminal = transitions["@isTerminatingState"]
                transitions.pop("@isTerminatingState")
                s = fsm.__name_to_state(name)
                for input, states in transitions.items():
                    s.transitions[input] = [fsm.__name_to_state(state) for state in states]
                if name == starting_state_name:
                    fsm.startingState = s
                if terminal:
                    fsm.terminalStates.add(s)
                else:
                    fsm.nonTerminalStates.add(s)
        return fsm

    def write_JSON(self, JSONFILE: str):
        """
        Takes JSON file name as string and writes the fsm to it
        """
        with open(JSONFILE, "w") as f:
            data = {'@startingState': self.__state_to_name(self.startingState)}
            for s in self.nonTerminalStates:
                data[self.__state_to_name(s)] = {"@isTerminatingState": False, 
                                                **{input : [self.__state_to_name(state) for state in states] for input, states in s}}
            for s in self.terminalStates:
                data[self.__state_to_name(s)] = {"@isTerminatingState": True, 
                                                **{input : [self.__state_to_name(state) for state in states] for input, states in s}}
            json.dump(data, f, indent=4, sort_keys=True)

    def __add_graph_edges(self, dot, state: State, visited : set["State"]):
        for key, value in state.transitions.items():
            for v in value:
                dot.edge(self.__state_to_name(state), self.__state_to_name(v), label=key)
                if v not in visited:
                    visited.add(v)
                    self.__add_graph_edges(dot, v, visited)

    def create_graph(self, label : str, filename : str):
        dot = graphviz.Digraph(label, filename=filename,format='png')
        dot.node_attr['shape'] = 'circle'
        dot.node('', shape='none')

        for state in self.nonTerminalStates:
            dot.node(self.__state_to_name(state), self.__state_to_name(state))

        for state in self.terminalStates:
            dot.node(self.__state_to_name(state), self.__state_to_name(state), shape='doublecircle')

        dot.edge('', self.__state_to_name(self.startingState), label='start')
        self.__add_graph_edges(dot, self.startingState, {self.startingState})
        dot.graph_attr['label'] = label
        dot.graph_attr['rankdir'] ='LR'
        dot.graph_attr['overlap'] = 'false'
        dot.graph_attr['splines'] = 'true'
        dot.graph_attr['sep'] = '+1'
        dot.graph_attr['nodesep'] = '1'
        dot.graph_attr['ranksep'] = '0.7 equally'
        dot.graph_attr['dpi'] = '100'
        dot.render(cleanup=True)


#### NFA Class
##### NFA is a subclass of FSM for Non-Deterministic Finite Automata, performs the following:
- Validates regex (raises value error if invalid)
- Creates an NFA from a regex if valid
- Is able to manipulate multiple NFAs to produce the result NFA using the following operations:
	- Alternation
	- Concatenation
	- Kleene Star
	- Kleene Plus
	- Optional

In [ ]:
class NFA(FSM):

    def __init__(self, regex : str = None, literal: str = None, single_char: str = None):
        """
        Creates an NFA from a regex string or a literal string or a single character
        """
        super().__init__(literal, single_char)
        self.regex = regex
        if literal is None and single_char is None and regex is not None:
            if NFA.__verify_regex(regex):
                output = NFA.__build_nfa(regex)
                self.startingState = output.startingState
                self.nonTerminalStates = output.nonTerminalStates
                self.terminalStates = output.terminalStates
                self.stateCounter = output.stateCounter
                self.stateToName = output.stateToName
                self.nameToState = output.nameToState
            else:
                raise ValueError("Invalid regex")

    @staticmethod
    def __verify_regex(regex):
        try:
            re.match(regex, "dummy1234")
        except re.error:
            return False
        else:
            if re.search(r"(^\/{1,})|(\/{2})$", regex):
                return False
            elif re.search(r"(^\|)|(\|$)|(\|{2,})", regex):
                return False
            else:
                return True

    @staticmethod
    def __build_nfa(regex: str) -> "NFA":
        """
        Converts a regex string to an NFA
        """
        i = 0
        # e.g: "ac(b|d)|e*" -> ['a', 'c', '(', 'b', '|', 'd', ')', '|', 'e', '*']
        regex : list[str] = [*regex]

        # Compute the NFA for each bracketed expression or character class
        # e.g:['a', 'c', '(', 'b', '|', 'd', ')', '|', 'e', '*']  -> ['a', 'c', NFA, '|', 'e', '*']
        while i < len(regex):
            if regex[i] == '(':
                open_bracket_index = i
                bracket_depth = 1
                while bracket_depth > 0:
                    i += 1
                    if regex[i] == '(':
                        bracket_depth += 1
                    elif regex[i] == ')':
                        bracket_depth -= 1
                modifying_fsm = NFA.__build_nfa(regex[open_bracket_index + 1:i])
                regex[open_bracket_index:i + 1] = [modifying_fsm]
                i = open_bracket_index
            if regex[i] == '[':
                open_bracket_index = i
                while regex[i] != ']':
                    i += 1
                modifying_fsm = NFA(single_char = ''.join(regex[open_bracket_index + 1:i]))
                regex[open_bracket_index:i + 1] = [modifying_fsm]
                i = open_bracket_index
            i += 1

        # Apply the unary operators
        # e.g: ['a', 'c', NFA, '|', 'e', '*'] -> ['a', 'c', NFA, '|', NFA]
        i = 0
        regex : list[str | "NFA"]
        while i < len(regex):
            if regex[i] in ['*', '+', '?']:
                if not isinstance(regex[i - 1], NFA):
                    regex[i - 1] = NFA(single_char = regex[i - 1])
                if regex[i] == '*':
                    regex[i - 1] = regex[i - 1].Zero_Or_More()
                elif regex[i] == '+':
                    regex[i - 1] = regex[i - 1].One_Or_More()
                elif regex[i] == '?':
                    regex[i - 1] = regex[i - 1].Optional()
                del regex[i]
                continue
            i += 1

        # Join contiguous literals to fsms
        # e.g: ['a', 'c', NFA, '|', NFA] -> [NFA, NFA, '|', NFA]
        i = 0
        starting_char_index = 0
        while i < len(regex):
            if isinstance(regex[i], NFA) or regex[i]  == '|':
                if i == starting_char_index:
                    starting_char_index += 1
                    i += 1
                else:
                    regex[starting_char_index:i] = [NFA(literal=''.join(regex[starting_char_index:i]))]
                    starting_char_index += 2
                    i = starting_char_index
            elif i == len(regex) - 1:
                regex[starting_char_index:] = [NFA(literal=''.join(regex[starting_char_index:]))]
                break
            else:
                i += 1

        # Concatenate the NFAs within the alternatives
        # e.g: [NFA, NFA, '|', NFA] -> [NFA, NFA]
        i = 0
        starting_nfa_index = 0
        nfas_to_alternate : list['NFA'] = []
        while i < len(regex):
            if regex[i] == '|':
                result_nfa = regex[starting_nfa_index]
                for nfa in regex[starting_nfa_index + 1:i]:
                    result_nfa = result_nfa.Concatenate(nfa)
                nfas_to_alternate.append(result_nfa)
                starting_nfa_index = i + 1
            elif i == len(regex) - 1:
                result_nfa = regex[starting_nfa_index]
                for nfa in regex[starting_nfa_index + 1:]:
                    result_nfa = result_nfa.Concatenate(nfa)
                nfas_to_alternate.append(result_nfa)
            i += 1

        # Apply the alternation
        return NFA.Alternate(nfas_to_alternate)

    # Override the FSM.read_JSON function to make it return an NFA object
    @staticmethod
    def read_JSON(JSONFILE: str) -> "NFA":
        """
        Reads an NFA from a JSON file
        """
        fsm = FSM.read_JSON(JSONFILE)
        result_nfa = NFA()
        result_nfa.startingState = fsm.startingState
        result_nfa.nonTerminalStates = fsm.nonTerminalStates
        result_nfa.terminalStates = fsm.terminalStates
        result_nfa.stateCounter = fsm.stateCounter
        result_nfa.stateToName = fsm.stateToName
        result_nfa.nameToState = fsm.nameToState
        return result_nfa


    def __epsilon_closure_rec(self, state : State, closure : set[State]) -> None:
        for char, next_states in state.transitions.items():
            if char == 'ε':
                for next_state in next_states:
                    if next_state not in closure:
                        closure.add(next_state)
                        self.__epsilon_closure_rec(next_state, closure)

    def epsilon_closure(self, states : list[State] | set[State]) -> FrozenSet[State]:
        closure : set[State] = set()
        closure.update(set(states))
        for state in states:
            self.__epsilon_closure_rec(state, closure)
        return frozenset(closure)

    def Concatenate(self, nfa: "NFA") -> "NFA":
        """
        Concatenates the current NFA with another NFA ***in place***\n
        i.e ( self . NFA )
        """
        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        # Adding the new non terminal states to the non terminal states
        self.nonTerminalStates.update(nfa.nonTerminalStates)
        # Updating the terminal states to the new ones
        self.terminalStates = nfa.terminalStates
        for state in past_terminal_states:
            if state.transitions.get("ε") is None:
                state.transitions["ε"] = [nfa.startingState]
            else:
                state.transitions["ε"].append(nfa.startingState)
        return self

    @staticmethod
    def Alternate(nfas: list["NFA"]) -> "NFA":
        """
        Alternates a list of NFAs ***in place***\n
        i.e ( nfas[0] | nfas[1] | ... | nfas[n] )
        """
        if len(nfas) == 1:
            return nfas[0]
        result_nfa = NFA()
        result_nfa.startingState = State()
        result_nfa.nonTerminalStates.add(result_nfa.startingState)
        new_terminating_state = State()
        result_nfa.terminalStates.add(new_terminating_state)

        # Mapping ε moves from new starting state to all the old starting states
        result_nfa.startingState.transitions["ε"] = [nfa.startingState for nfa in nfas]

        for nfa in nfas:
            # Adding the all old states to the new non terminal states
            result_nfa.nonTerminalStates.update(nfa.nonTerminalStates)
            result_nfa.nonTerminalStates.update(nfa.terminalStates)
            # Mapping epislon moves from old terminal states to new terminal state
            for state in nfa.terminalStates:
                if not state.transitions.get("ε"):
                    state.transitions["ε"] = [new_terminating_state]
                else:
                    state.transitions["ε"].append(new_terminating_state)
        return result_nfa

    def Zero_Or_More(self) -> "NFA":
        """
        Applies Kleene star to the current NFA ***in place***\n
        i.e ( self* )
        """
        new_starting_state = State()
        new_terminating_state = State()

        # Mapping epislon moves from new starting state to old starting state and new terminating state
        new_starting_state.transitions["ε"] = [self.startingState, new_terminating_state]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates.add(new_starting_state)
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state and new starting state
        for state in past_terminal_states:
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state, self.startingState]
            else:
                state.transitions["ε"].extend([new_terminating_state, self.startingState])
        return self

    def One_Or_More(self) -> "NFA":
        """
        Applies Kleene plus to the current NFA ***in place***\n
        i.e ( self+ )
        """
        new_starting_state = State()
        new_terminating_state = State()

        # Mapping epislon moves from new starting state to old starting state
        new_starting_state.transitions["ε"] = [self.startingState]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates.add(new_starting_state)
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state and new starting state
        for state in past_terminal_states:
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state, self.startingState]
            else:
                state.transitions["ε"].extend([new_terminating_state, self.startingState])
        return self

    def Optional(self) -> "NFA":
        """
        Applies question mark operator to the current NFA ***in place***\n
        i.e ( self? )
        """
        new_starting_state = State()
        new_terminating_state = State()

        # Mapping epislon moves from new starting state to old starting state and new terminating state
        new_starting_state.transitions["ε"] = [self.startingState, new_terminating_state]

        past_terminal_states = self.terminalStates
        # Adding the old terminal states to the non terminal states
        self.nonTerminalStates.update(past_terminal_states)
        self.nonTerminalStates.add(new_starting_state)
        # Updating the starting state to the new one
        self.startingState = new_starting_state
        # Updating the terminal states to the new one
        self.terminalStates = {new_terminating_state}

        # Mapping epislon moves from old terminal states to new terminal state
        for state in past_terminal_states:
            if not state.transitions.get("ε"):
                state.transitions["ε"] = [new_terminating_state]
            else:
                state.transitions["ε"].append(new_terminating_state)
        return self

#### Interpreting ranges 😲🤓

In [ ]:
# test_str = "A-Za-z0-9._&+@"
# test_str = "a-zA-Z0-9_$-"
test_str = "a-zA-Z0-9-_"
 
# printing original string
print("The original string is : " + test_str)
 
# Convert String ranges to list
# Using for loop and string manipulation
def listFromRange(rng : str) -> list[str]:
    res = []
    for i in range(0, len(rng), 3):
        s = rng[i:i+3]
        if s[1] == '-' and (s[0::2].isdecimal() or s[0::2].isalpha()):
            start, end = map(int, [ord(char) for char in s.split('-')])
            res.append(chr(start))
            for i in range(start+1, end+1):
                res.append(chr(i))
        else:
            if len(s) == 1:
                res.append(s)
            else:
                res.extend([*s])
    return res
# printing result
print("List after conversion from string : " + str(listFromRange(test_str)))

#### DFA Class:
##### DFA is a subclass of FSM for Deterministic Finite Automata, performs the following:
- Creates a DFA from an NFA
- Minimizes a DFA using State Minimzation Algorithm (Optional, see constructor arguments)

In [ ]:
class DFA(FSM):

    def __init__(self, nfa : NFA, minimize : bool = True):
        """
        Creates a minimized DFA from the given NFA
        """
        super().__init__()
        self.DFAStateToNFAStates : dict[State, FrozenSet[State]] = dict()
        self.NFAStatesToDFAState : dict[FrozenSet[State], State] = dict()
        self.DFAStateToMinDFAState : dict[State, State] = dict()
        self.MinDFAStateToDFAStates : dict[State, set[State]] = dict()
        self.__convertNFA2DFA(nfa)
        if minimize:
            self.__minimize()

    def __nfa_states_to_dfa_state(self, nfa_states : FrozenSet[State]) -> State:
        """
        Returns the DFA state corresponding to the given NFA state set\n
        Returns a new state if no state is assigned to the given set
        """
        # Does the set already have a state?
        if nfa_states in self.NFAStatesToDFAState:
            # Return the state
            return self.NFAStatesToDFAState[nfa_states]
        # Assign a new state to the set
        self.NFAStatesToDFAState[nfa_states] = State()
        # Update the reverse mapping
        self.DFAStateToNFAStates[self.NFAStatesToDFAState[nfa_states]] = nfa_states
        return self.NFAStatesToDFAState[nfa_states]

    def __dfa_state_to_nfa_states(self, dfa_state : State) -> FrozenSet[State]:
        """
        Returns the set of NFA states corresponding to the given DFA state
        """
        return self.DFAStateToNFAStates[dfa_state]

    def __convertNFA2DFA(self, nfa : NFA):
        """
        Converts the given NFA to a DFA
        """
        # The starting state of the DFA is the set of states reachable from the starting state of the NFA by epsilon moves
        starting_state = self.__nfa_states_to_dfa_state(nfa.epsilon_closure([nfa.startingState]))
        # The starting state of the DFA is the starting state of the NFA
        self.startingState = starting_state
        # The set of states to process
        states_to_process = {starting_state}
        # While there are states to process
        while states_to_process:
            # Get the next state to process
            state = states_to_process.pop()
            # Get the set of NFA states corresponding to the current DFA state
            nfa_states = self.__dfa_state_to_nfa_states(state)
            transitions : dict[str, set[State]] = {}
            isTerminal : bool = False
            for nfa_state in nfa_states:
                if nfa_state in nfa.terminalStates:
                    isTerminal = True
                for char, next_states in nfa_state.transitions.items():
                    if char == "ε":
                        continue
                    if char not in transitions:
                        transitions[char] = set()
                    transitions[char].update(next_states)

            for char, next_states in transitions.items():
                next_states = nfa.epsilon_closure(next_states)
                next_state = self.__nfa_states_to_dfa_state(next_states)
                if char not in state.transitions:
                    state.transitions[char] = [next_state]
                    states_to_process.add(next_state)
            if isTerminal:
                self.terminalStates.add(state)
            else:
                self.nonTerminalStates.add(state)
        self.NFAStatesToDFAState = dict()
        self.DFAStateToNFAStates = dict()

    def __dfa_states_to_min_dfa_state(self, dfa_states : set[State]) -> State:
        """
        Returns the minimized DFA state corresponding to the given set of DFA states\n
        Creates one if no state is assigned to the given set\n
        ***Assumes the given set have the same representative state***
        """
        min_state = None
        # Does the set already have a state?
        for dfa_state in dfa_states:
            if dfa_state in self.DFAStateToMinDFAState:
                break
        else:
            min_state = State()
        if min_state is None:
            min_state = self.DFAStateToMinDFAState[dfa_state]
        for dfa_state in dfa_states:
            self.DFAStateToMinDFAState[dfa_state] = min_state
        self.MinDFAStateToDFAStates[min_state] = dfa_states
        return min_state

    def __dfa_state_to_min_dfa_state(self, dfa_state : State) -> State:
        """
        Returns the minimized DFA state corresponding to the given DFA state\n
        Creates one if no state is assigned to the given state
        """
        if dfa_state not in self.DFAStateToMinDFAState:
            self.DFAStateToMinDFAState[dfa_state] = State()
            self.MinDFAStateToDFAStates[self.DFAStateToMinDFAState[dfa_state]] = {dfa_state}
            return self.DFAStateToMinDFAState[dfa_state]
        return self.DFAStateToMinDFAState[dfa_state]

    def __min_dfa_state_to_dfa_states(self, min_dfa_state : State) -> set[State]:
        """
        Returns the set of minimized DFA states corresponding to the given DFA state
        """
        return self.MinDFAStateToDFAStates[min_dfa_state]

    def __min_dfa_state_to_a_dfa_state(self, min_dfa_state : State) -> State:
        """
        Returns a DFA state corresponding to the given minimized DFA state
        """
        return next(iter(self.MinDFAStateToDFAStates[min_dfa_state]))

    def __clear_min_dfa_state(self, min_dfa_state : State):
        """
        Removes the given minimized DFA state and its corresponding DFA states dictionary bindings
        """
        for dfa_state in self.MinDFAStateToDFAStates[min_dfa_state]:
            del self.DFAStateToMinDFAState[dfa_state]
        del self.MinDFAStateToDFAStates[min_dfa_state]

    def __states_are_equivalent(self, state1 : State, state2 : State) -> bool:
        """
        Returns whether the given states are equivalent\n
        i.e they have transitions to the same state set for the same input symbols
        """
        if len(state1.transitions) != len(state2.transitions):
            return False
        for char, state_set1 in state1.transitions.items():
            if char not in state2.transitions:
                return False
            state_set1 = self.__dfa_state_to_min_dfa_state(state_set1[0])
            state_set2 = self.__dfa_state_to_min_dfa_state(state2.transitions[char][0])
            if state_set1 != state_set2:
                return False
        return True

    def __set_min_dfa_state_for_state(self, dfa_state : State, min_dfa_state : State):
        """
        Sets the minimized DFA state for the given DFA state
        """
        self.DFAStateToMinDFAState[dfa_state] = min_dfa_state
        self.MinDFAStateToDFAStates[min_dfa_state].add(dfa_state)

    def __set_min_dfa_state_transitions(self, min_dfa_state : State):
        """
        Sets the transitions for the given minimized DFA state
        """
        dfa_state = self.__min_dfa_state_to_a_dfa_state(min_dfa_state)
        for char, next_states in dfa_state.transitions.items():
            min_dfa_state.transitions[char] = [self.__dfa_state_to_min_dfa_state(next_states[0])]


    def __minimize(self):
        """
        Minimizes the DFA
        """
        # step 1: partition the states into two sets: terminal and non-terminal
        minimized_states : set[State] = set()
        minimized_states.add(self.__dfa_states_to_min_dfa_state(self.terminalStates))
        minimized_states.add(self.__dfa_states_to_min_dfa_state(self.nonTerminalStates))
        # loop for step 3, iterate on the partitioned states if changed
        changes_occured = True
        while changes_occured:
            changes_occured = False
            states_to_process : set[State] = set(minimized_states)
            # step 2: for each partition, check if it can be split into two partitions
            for min_state in states_to_process:
                dfa_states = self.__min_dfa_state_to_dfa_states(min_state)
                self.__clear_min_dfa_state(min_state)
                new_min_states : set[State] = set()
                # step 2.a: for each state in the partition, split if not equivalent
                for state in dfa_states:
                    for new_min_state in new_min_states:
                        state_to_compare_against = self.__min_dfa_state_to_a_dfa_state(new_min_state)
                        if self.__states_are_equivalent(state, state_to_compare_against):
                            self.__set_min_dfa_state_for_state(state, new_min_state)
                            break
                    else:
                        new_min_states.add(self.__dfa_state_to_min_dfa_state(state))
                # step 2.b: if the partition was split, update the partitioned states
                minimized_states.remove(min_state)
                minimized_states.update(new_min_states)
                if len(new_min_states) > 1:
                    changes_occured = True
        # Clear data structures used for minimization
        self.DFAStateToMinDFAState = dict()
        self.MinDFAStateToDFAStates = dict()
        # step 4: update the DFA
        terminalStates  : set[State] = set()
        nonTerminalStates  : set[State] = set()
        # step 4.a: set the starting state to the minimized starting state
        self.startingState = self.__dfa_state_to_min_dfa_state(self.startingState)
        # step 4.b: set the transitions and terminal/non-terminal states
        for min_state in minimized_states:
            self.__set_min_dfa_state_transitions(min_state)
            dfa_state = self.__min_dfa_state_to_a_dfa_state(min_state)
            if dfa_state in self.terminalStates:
                terminalStates.add(min_state)
            else:
                nonTerminalStates.add(min_state)
        self.terminalStates = terminalStates
        self.nonTerminalStates = nonTerminalStates


#### Examples:

In [ ]:
nfa = NFA('ab(b|c)*d+')
nfa.create_graph('NFA', 'NFA')
dfa = DFA(nfa)
dfa.create_graph('DFA', 'DFA')